## scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 비동기 방식으로 데이터 수집 > 데이터 수집 속도가 빠름
    - 멀티스레딩으로 데이터 수집

In [1]:
# install scrapy
#!pip install scrapy

### Gmarket 베스트 200 상품 데이터 수집

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [2]:
# 1. 스크레피 프로젝트 생성
# scrapy startproject 'project name': project dir + 코드 생성

In [ ]:
!scrapy startproject gmarket

In [ ]:
!tree gmarket /f

scrapy 프로젝트 내 파일 설명
- items.py : 수집할 데이터의 구조 정의
- middewares : 데이터를 수집할 때 headers 정보와 같은 내용을 설정
- pipelines : 데이터를 수집한 후에 코드 실행 정의
- settings : 크롤링 설정: robots.txt 규칙 준수 여부, 크롤링 시간 간격 등을 설정
- spiders : 디렉토리 : 크롤링 절차 정의

In [6]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터

In [2]:
# 링크 데이터 수집
url = 'http://corners.gmarket.co.kr/Bestsellers'
request = requests.get(url)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [3]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
# @href 속성값 가져오기

links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953135767395634',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953135767395634']

In [4]:
# 상세 데이터 : 상품명, 가격
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

# 결과 데이터는 항상 리스트 형태 -> [0] 선택
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract() 
price = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [14]:
# 3. items.py : 코드 작성 : model(데이터의 구조: 수집할 데이터의 컬럼 정의)

In [6]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [16]:
# 4. spider.py : 코드 작성 : 크롤링 절차 정의

In [5]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB" # 프로젝트 실행할 때 사용할 spider 이름
    allow_domain = ["gmarket.co.kr"] # 크롤링할 도메인 지정
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"] # 최초에 request/respose할 url
    
    # 처음에 request하는 함수는 이름 고정
    def parse(self, response): # start_urls에서 받은 response를 받아서 수행
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content) # yield - 실행될 때마다 return data 달라짐
    
    # 이후 사용되는 함수는 이름 자유 -> scrapy.Request()내 callback과 같아야함
    def parse_content(self, response): # parse에서 받은 response를 받아서 수행
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [18]:
# 5. 스크레피 프로젝트 실행
# cd: change directory / -o: output
# 터미널 -> gmarket 디렉토리에서 아래의 커멘드 실행 
# scrapy crawl GMB -o items.csv

In [ ]:
%ls gmarket/

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("gmarket/items.csv")
df.head()

,link,price,title
0,http://item.gmarket.co.kr/Item?goodscode=25196...,"25,030",브라운 면도기 세정액 CCR5+1
1,http://item.gmarket.co.kr/Item?goodscode=20396...,"49,900",1+1 2개 무선 선풍기 탁상용 미니 휴대용 캠핑용 벽걸이 사무실 선물용 가정용
2,http://item.gmarket.co.kr/Item?goodscode=24572...,"12,000",레고 닌자고 브릭마스터 외 6종 선택구매
3,http://item.gmarket.co.kr/Item?goodscode=19279...,"23,900",양꼬치 양깍두기 양고기 1kg + 쯔란1봉
4,http://item.gmarket.co.kr/Item?goodscode=16359...,"9,900",아삭아삭 미니 오이 20개 /2개주문시 50개발송
